In [1]:
%env CUDA_VISIBLE_DEVICES=4

env: CUDA_VISIBLE_DEVICES=4


In [2]:
from magma import Magma
from magma.image_input import ImageInput
from activations.torch import Rational
from magma.adapters import Adapter
import random
import torch
from tqdm import tqdm
import copy, json 

In [3]:
plain_magma = Magma.from_checkpoint(
    checkpoint_path='./../model_checkpoints/max_checkpts/upload/step_21500/mp_rank_00_model_states.pt',
                                    config_path='./../fb20-dgx2-configs/Magma_Plain.yml')

Loading GPTJ language model...
From EleutherAI/gpt-J-6B
loading magma checkpoint from: ./../model_checkpoints/max_checkpts/upload/step_21500/mp_rank_00_model_states.pt
magma successfully loaded


In [4]:
# model = Magma(config='./../fb20-dgx2-configs/dev.yml').cuda()
model = Magma.from_checkpoint(
    checkpoint_path='./../model_checkpoints/switch2_7250.pt',
    config_path='./../fb20-dgx2-configs/dev.yml').cuda()

Loading GPTJ language model...
From EleutherAI/gpt-j-6B


KeyboardInterrupt: 

In [6]:
import os, sys
aokvqa_dir = '/home/ml-mmeuer/adaptable_magma/magma/datasets/aokvqa/'
sys.path.append(os.path.abspath(os.path.join('..', 'aokvqa')))
from aokvqa.load_aokvqa import load_aokvqa, get_coco_path
from aokvqa.evaluation.eval_predictions import eval_aokvqa
eval_set = load_aokvqa(aokvqa_dir, 'val') 

In [18]:
os.path.join(okvqa_dir, f'OpenEnded_mscoco_{mode}2014_questions.json')

'/home/ml-mmeuer/adaptable_magma/magma/datasets/aokvqa/OpenEnded_mscoco_val2014_questions.json'

In [7]:
ok_vqa_dir = '/home/ml-mmeuer/adaptable_magma/magma/datasets/OK_VQA/'
mode = 'val'
ok_vqa_val = json.load(open(os.path.join(ok_vqa_dir, f'OpenEnded_mscoco_{mode}2014_questions.json')))
annotations = {}
annotations_raw = json.load(open(f'/home/ml-mmeuer/adaptable_magma/magma/datasets/OK_VQA/mscoco_val2014_annotations.json'))

for a in annotations_raw['annotations']:
    answers = [ans['answer'] for ans in a['answers'] ]
    annotations[a['question_id']] = {'answers': answers}

In [11]:
def get_coco_path_2014(dir, id, mode): 
    return os.path.join(dir , f'COCO_{mode}2014_{id:012}.jpg')
    

In [16]:
questions = json.load(open('/home/ml-mmeuer/adaptable_magma/magma/datasets/OK_VQA/OpenEnded_mscoco_val2014_questions.json'))
coco_path = '/home/ml-mmeuer/adaptable_magma/magma/datasets/coco/val2014/'
model_for_run = plain_magma.cuda()
for q in questions['questions']:
    print(get_coco_path_2014(coco_path, q['image_id'], 'val'))
    image = ImageInput(get_coco_path_2014(coco_path, q['image_id'], 'val'))
    prompt = q['question']
    
    inputs =[
    ## supports urls and path/to/image
    image,
    prompt
    ]   
    embeddings = model_for_run.preprocess_inputs(inputs).cuda()
    # print('Answer: ', end='')
    answer = model_for_run.generate(
        embeddings = embeddings,
        max_steps = 6,
        temperature = 0.7,
        top_k = 0,
    )[0]   
    print(answer)
def eval_okvqa(dataset, preds):
    acc = []
    for q in dataset.keys(): 
        if q not in eval_set.keys(): 
            acc.append(0.0)
            continue
        pred = preds[q]
        choices = dataset

/home/ml-mmeuer/adaptable_magma/magma/datasets/coco/val2014/COCO_val2014_000000297147.jpg
 hd bike on a stand
/home/ml-mmeuer/adaptable_magma/magma/datasets/coco/val2014/COCO_val2014_000000339761.jpg


RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

In [7]:
def prompt_element(d, context=None, include_choices=False, answer=False):
    return  f"Q: {d['question']}\n" + \
           (f"Choices: {', '.join(d['choices'])}.\n" if include_choices else '') + \
            f"A:" + (f" {d['choices'][d['correct_choice_idx']]}" if answer else '')

In [ ]:
def eval_aokvqa(dataset, preds, multiple_choice=False, strict=True):
    


In [9]:
model_for_run = plain_magma.cuda()
exp_setup = {}
coco_dir = '/home/ml-mmeuer/adaptable_magma/magma/datasets/coco'
predictions = {}
for d in tqdm(eval_set):
    q = d['question_id']
    prompt = prompt_element(d, include_choices=True)
    image = ImageInput(get_coco_path('val', d['image_id'], coco_dir = coco_dir))
    inputs =[
    ## supports urls and path/to/image
    image,
    prompt
    ]   
    embeddings = model_for_run.preprocess_inputs(inputs)
    # print('Answer: ', end='')
    answer = model_for_run.generate(
        embeddings = embeddings,
        max_steps = 6,
        temperature = 0.7,
        top_k = 0,
    )[0] 
    predictions[q] = answer

eval_aokvqa_score = eval_aokvqa(eval_set, exp_setup['predictions'], multiple_choice=True,strict=False)
exp_setup['eval_aokvqa_score'] = eval_aokvqa_score.detach().cpu().numpy()

100%|██████████| 1145/1145 [04:32<00:00,  4.20it/s]


AttributeError: 'float' object has no attribute 'detach'

In [10]:
eval_aokvqa_score

0.0

In [16]:
exp_log = {}
run_idx = 1
runs = 5 
for hard_switch in [True, False]:
    for on_with_identity in [True, False]:
        for run in range(runs):
            model_for_run = copy.deepcopy(model) 
            exp_setup = {
                'hard_switch'  : hard_switch,
                'switch':{}
            }
            n_adapters = 0
            for n , m in model_for_run.named_modules():
                if isinstance(m, Adapter):
                    
                    probs = m.switch_logits
                    rendn = random.random()
                    turn_adapter_on = rendn < probs[0]
                    
                    exp_setup['switch'][n] = turn_adapter_on.item()
                    if turn_adapter_on:

                        if on_with_identity: 
                            m.adapter_switch = False
                        else: 
                            if hard_switch:
                                m.fixed_idx = 0
                                m.training = False
                            m.adapter_switch = True
                            m.training = True
                        n_adapters += 1
                    else: 

                        m.training = False
                        m.fixed_idx = 1


            exp_setup['n_adapters'] = n_adapters
            exp_setup['predictions'] = {}
            coco_dir = '/home/ml-mmeuer/adaptable_magma/magma/datasets/coco'
            for d in tqdm(eval_set):
                q = d['question_id']
                prompt = prompt_element(d, include_choices=True)
                image = ImageInput(get_coco_path('val', d['image_id'], coco_dir = coco_dir))
                inputs =[
                ## supports urls and path/to/image
                image,
                prompt
                ]   
                embeddings = model_for_run.preprocess_inputs(inputs)
                # print('Answer: ', end='')
                answer = model_for_run.generate(
                    embeddings = embeddings,
                    max_steps = 6,
                    temperature = 0.7,
                    top_k = 0,
                )[0] 
                exp_setup['predictions'][q] = { 'multiple_choice': answer}
            run_idx += 1
            eval_aokvqa_score = eval_aokvqa(eval_set, exp_setup['predictions'], multiple_choice=True,strict=False)
            exp_setup['eval_aokvqa_score'] = eval_aokvqa_score.detach().cpu().numpy()
            run_name = f'Run_{run_idx}_Hard_Switch{hard_switch}_N_Adapter_{n_adapters}_Identity_{on_with_identity}'
            with open(f'./aokvqa_runs/{run_name}.json', 'w') as f:
                json.dump(exp_setup, f)
            
        

100%|██████████| 1145/1145 [04:41<00:00,  4.07it/s]


TypeError: Object of type Tensor is not JSON serializable

In [27]:
for k, i in exp_setup.items():
    print(k, i)

hard_switch True
switch {'lm.transformer.h.0.mlp.1': tensor(False, device='cuda:0'), 'lm.transformer.h.1.mlp.1': tensor(False, device='cuda:0'), 'lm.transformer.h.2.mlp.1': tensor(False, device='cuda:0'), 'lm.transformer.h.3.mlp.1': tensor(False, device='cuda:0'), 'lm.transformer.h.4.mlp.1': tensor(False, device='cuda:0'), 'lm.transformer.h.5.mlp.1': tensor(False, device='cuda:0'), 'lm.transformer.h.6.mlp.1': tensor(True, device='cuda:0'), 'lm.transformer.h.7.mlp.1': tensor(False, device='cuda:0'), 'lm.transformer.h.8.mlp.1': tensor(False, device='cuda:0'), 'lm.transformer.h.9.mlp.1': tensor(False, device='cuda:0'), 'lm.transformer.h.10.mlp.1': tensor(False, device='cuda:0'), 'lm.transformer.h.11.mlp.1': tensor(False, device='cuda:0'), 'lm.transformer.h.12.mlp.1': tensor(False, device='cuda:0'), 'lm.transformer.h.13.mlp.1': tensor(False, device='cuda:0'), 'lm.transformer.h.14.mlp.1': tensor(False, device='cuda:0'), 'lm.transformer.h.15.mlp.1': tensor(False, device='cuda:0'), 'lm.transf

In [ ]:
print(eval_aokvqa(eval_set, exp_setup['predictions'], multiple_choice=True, strict=False))

0.0
